# Laboratorio 1, High Performance Computing
Con el fin de realizar un repaso de las habilidades con el lenguaje ***C*** se debe construir un programa que permita multiplicar 2 matrices, teniendo en cuenta las siguientes condiciones :

* Las dos Matrices se leeran de dos archivos de texto separados por comas. 
* los dos primeras lineas contienen la cantidad de filas y columnas de la matriz respectiva. 

*Ejemplo:* 
```
2
2
1,2
3,4
```
El resulta de la multiplicación de las dos matrices debe ser enviado via stdout aun archivo de texto,
utilizar ***Taller1HPC*** es el asunto que debe tener el email enviado al docente John.

<cite>Como tipo de dato se utiliza precisión sencilla ***float***</cite>
# Solución

## Multiplicación de Matrices
dadas dos matrices $A$,$B$ de la forma:
$$A = \begin{pmatrix}
 a_{1 1} & \cdots & a_{1 n} \\
 \vdots & \ddots & \vdots \\
 a_{m 1} & \cdots & a_{m n}
 \end{pmatrix}, B = \begin{pmatrix}
 b_{1 1} & \cdots & b_{1 n} \\
 \vdots & \ddots & \vdots \\
 b_{m 1} & \cdots & b_{m n}
 \end{pmatrix}
 $$

Escritas en los textos como $A:=(a_{i j})_{m \times n}$, $B:=(b_{i j})_{n \times p}$, donde $m,n,p$ indican las filas y columnas de cada matriz, el producto de $A\cdot B$ es:
$C = AB_{}^{}$.

$$ \begin{pmatrix}
 a_{1 1} & \cdots & a_{1 n} \\
 \vdots & \ddots & \vdots \\
 a_{m 1} & \cdots & a_{m n}
 \end{pmatrix} \cdot \begin{pmatrix}
 b_{1 1} & \cdots & b_{1 p} \\
 \vdots & \ddots & \vdots \\
 b_{n 1} & \cdots & b_{n p}
 \end{pmatrix}$$
 
 $$\begin{pmatrix}
 a_{11}b_{11}+ \cdots +a_{1n}b_{n1} & \cdots & a_{11}b_{1p}+ \cdots +a_{1n}b_{np} \\
 \vdots & \ddots & \vdots \\
 a_{m1}b_{11}+ \cdots +a_{mn}b_{n1} & \cdots & a_{m1}b_{1p}+ \cdots +a_{mn}b_{np}
 \end{pmatrix}
$$

Que no es mas que la sumatoria de multiplicar la fila por la columna para cada elemento de la matriz resulta:
$$c_{ij} = \sum_{r=1}^n a_{ir}b_{rj}$$


*Nota[!]:La cantindad de Columnas debe ser igual a la cantidad de filas de la segunda matriz,$A:=(a_{i j})_{m \times n}$, $B:=(b_{i j})_{n \times p}$, tendra como resultado una Matriz $B:=(b_{i j})_{m \times p}$*.

De las observaciones anteriores podemos decir que  para resolver la multiplicación de las matrices $A\cdot B$ es necesario realizar $m*n*p$ multiplicaciones, Seria posible utilizar una gran cantidad de tecnicas como se describe  en [1][1] y [2][2], pero por tiempo utilizare la version interactiva tiene un costo $Θ(n^{3})$ como se muestra en [3][3]:

<img src="http://i.imgur.com/Y4OmGFt.png" height="650" width="640">

## Codigo implementado
Para resolver este problema he hecho uso de los Makefiles para hacer parseo de los datos que  se me entrega, 
la ejecucion de este programa obligatoriamente establece que los archivos de entrada tengan los nombre ***data1.txt***, ***data2.txt***, siendo *data1.txt* el archivo que contiene los datos reales de la Matriz $A$, y  lo mismo para $B$, 
luego lo unico que sera hacaer es:
**make run**

### Lectura de Archivo
Para lograr la lectura de los archivo en lenguaje **C** utilizamos las funciones fopen,fwrite para la apertura del archivo, y el almacenamiento de este, el siguiente fragmento de codigo en C hace uso de fopen para abrir el archivo:
```c
#include <stdio.h>
int main(){
  FILE *archivo;
  archivo = fopen("data1.txt","r");
  fclose(fclose);
  return 0;
  }
```
Con el ejemplo anterior tenemos la posibilidad de acceder a los datos del archivo, ahora para la lectura hacemos uso de la funcion **fscanf()** que tiene los mismos parametros de **scanf()**, utilizando las mismas opciones para los diferentes tipos de dato. `int fscanf(FILE *stream, const char *format, ...)`. de los archivos recibidos por el programa, nos damos cuenta que las dos primeras lineas siempre serán enteros, por lo cual simplemente se leen cuatro enteros indicando filas, y columnas de cada matriz:

```C
  //previous version has conflict with the pointer movmnt.
  fscanf(archi,"%i",&rows1);
  fscanf(archi,"%i",&cols1);
  fscanf(archi2,"%i",&rows2);
  fscanf(archi2,"%i",&cols2);
    float **MatA = (float **)calloc(rows1,sizeof(float*));
  for(i = 0; i < rows1; i++)
      MatA[i] = (float *)calloc(cols1 ,sizeof(float));
```

Siguiendo el algoritmo iterativo anterior validamos que las columnas de la primera matriz sean iguales a las filas de la segunda matriz, de lo contrario saqueremos un mensaje de *Dimensiones incompatibles*. 

Uno de los temas mas delicados a la hora de realizar este ejercicio fue el de manejar la memoria de manera dinamica, el lenguaje C nos da mucho control para hacer uso de esta, mediante un mecanismo muy simple, solicitud de memoria(tipicamente la parte del *heap*) y liberacion de la misma. El ciclo es sencillo, cuando se precisa almacenar un nuevo dato, se solicita tanta memoria en **bytes** como sea necesaria, y una vez que ese dato ya no se necesita la memoria se devuelve para poder ser reutilizada, para solicitar memoria se utiliza la funcion malloc o calloc en mi caso dado que inicializa los datos de la memoria solicitada con valores de ceros.


```C
  float **MatA = (float **)calloc(rows1,sizeof(float*));
  for(i = 0; i < rows1; i++)
      MatA[i] = (float *)calloc(cols1 ,sizeof(float));
```
...to be continue go to sleep

### Ejecución
Una vez se tiene garantizado los nombres de los archivos de entrada, utilizando la utilidad ***sed*** reemplazamos las comas en ambos archivos, luego nuestro programa en una de la secciones del Makefile tiene :

./solve data1.txt data2.txt 

```c
#include <stdio.h>
#include <stdlib.h>
//High Performance Computing
//Lab#1: Matrix Multiplication 
//hfjimenez@utp.edu.co, 2017-2
...to be continued


```
#### Referencias :
- https://en.wikipedia.org/wiki/Matrix_multiplication_algorithm#Iterative_algorithm
- https://es.wikibooks.org/wiki/Optimizaci%C3%B3n_del_Producto_de_Matrices
[1]: https://en.wikipedia.org/wiki/Matrix_multiplication_algorithm#Iterative_algorithm
[2]: https://es.wikibooks.org/wiki/Optimizaci%C3%B3n_del_Producto_de_Matrices
[3]: https://en.wikipedia.org/wiki/Matrix_multiplication_algorithm#Iterative_algorithm